## Summary

Use this notebook to delete lingering pipelines from GitLab.

---

## Imports

In [ ]:
import json
import os
import shlex
import subprocess

## Parameters

In [ ]:
PROJECT_ID = 21481523

In [ ]:
GITLAB_AUTH_TOKEN = os.environ['GITLAB_AUTH_TOKEN']

In [ ]:
PIPELINES_TO_KEEP = [229764931]

## Workspace

In [ ]:
def get_pipeline_infos():
    proc = subprocess.run(
        shlex.split(
            f'curl --header "PRIVATE-TOKEN: {GITLAB_AUTH_TOKEN}" '
            f"https://gitlab.com/api/v4/projects/{PROJECT_ID}/pipelines?per_page=100"
        ),
        check=True,
        capture_output=True,
    )
    pipeline_infos = json.loads(proc.stdout)
    return pipeline_infos


def get_pipeline_ids(pipeline_infos):
    pipeline_ids = [
        pipeline_id
        for pipeline_info in pipeline_infos
        if (pipeline_id := pipeline_info["id"]) not in PIPELINES_TO_KEEP
    ]
    return pipeline_ids


def delete_pipelines(pipeline_ids):
    for pipeline_id in pipeline_ids:
        proc = subprocess.run(
            shlex.split(
                f'curl --header "PRIVATE-TOKEN: {GITLAB_AUTH_TOKEN}" '
                '--request "DELETE" '
                f"https://gitlab.com/api/v4/projects/{PROJECT_ID}/pipelines/{pipeline_id}"
            ),
            check=True,
        )

In [ ]:
pipeline_ids = None
while pipeline_ids is None or len(pipeline_ids) > 0:
    pipeline_infos = get_pipeline_infos()
    pipeline_ids = get_pipeline_ids(pipeline_infos)
    print(len(pipeline_ids))
    delete_pipelines(pipeline_ids)